In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/train_3PIRKSI.zip", 'r')
zip_ref.extractall("/content/")
zip_ref.close()

In [0]:
import pandas as pd
import numpy as np
data=pd.read_csv("/content/train.csv")

In [0]:
data = data.fillna(method="ffill")

In [0]:
# total number of words
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

184506

In [0]:
data.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [0]:
# total number of tags in the training data
data_tags = list(set(data["tag"].values))
n_tags = len(data_tags); n_tags


3

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID ").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          


In [0]:
# some preprocessing of the dataset
# 1) remove punctuations and stopwords

import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=stopwords.words('english')
print(type(stop))
rem_row_list=[]
data_rem=data[data['Word'].map(lambda x: x not in stop)]


data_rem.shape

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<class 'list'>


(3278889, 5)

In [0]:
# total number of words
words = list(set(data_rem["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words




184371

In [0]:
# 2) remove punctuations:
import string
data_rem['Word'].str.replace('[{}]'.format(string.punctuation), str(np.nan))
data1=data_rem[data_rem['Word'].map(lambda x: len(str(x))>1)]
data1.head(8)



,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
2,3,1,1,Low-,O
4,5,1,1,Middle-Income,O
5,6,1,1,Countries,O
7,8,1,1,Burden,O
9,10,1,1,Drivers,O
12,13,1,1,Emerging,O
13,14,1,1,Challenges,O


In [0]:
data1.shape
data2=data1
data2.shape
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
l=pos_tag(data2['Word'].tolist())
words, tags = zip(*l)
data2['POS'] = pd.Series(tags)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
data2.head()

,id,Doc_ID,Sent_ID,Word,tag,POS
0,1,1,1,Obesity,O,NNP
2,3,1,1,Low-,O,NNP
4,5,1,1,Middle-Income,O,NNP
5,6,1,1,Countries,O,NNP
7,8,1,1,Burden,O,NNP


In [0]:
print(data2.columns)
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby(["Sent_ID"]).apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data2)
sent = getter.get_next()
sentences=getter.sentences



Index(['id', 'Doc_ID', 'Sent_ID', 'Word', 'tag', 'POS'], dtype='object')


In [0]:
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(data_tags)}
print(word2idx["Countries"])
print(data2['tag'].value_counts())


2440918
O                2524903
B-indications      52842
I-indications      42121
Name: tag, dtype: int64


In [0]:

from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)

Using TensorFlow backend.


In [0]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [0]:
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [0]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)


In [0]:
# setup lstm crf architecture
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output
model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 20)            3687440   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 75, 3)             168       
Total params: 3,721,058
Trainable params: 3,721,058
Non-trainable params: 0
_________________________________________________________________


In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-oeai8itb
  Stored in directory: /tmp/pip-ephem-wheel-cache-xn7bp_ve/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=3,
                    validation_split=0.1, verbose=1)


In [1]:

hist = pd.DataFrame(history.history)
import pickle
with open('model', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


"\nhist = pd.DataFrame(history.history)\nimport pickle\nwith open('model', 'wb') as file_pi:\n        pickle.dump(history.history, file_pi)\n"

In [0]:
!pip install seqeval
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
test_pred = model.predict(X_te, verbose=1)

In [0]:
idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))